In [1]:
import pandas as pd

In [2]:
#pip install git+https://github.com/quantopian/pyfolio
# import again if first time not work
import pyfolio

C:\Users\Andy\PycharmProjects\finrlpaper1\Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020\finrlpaperv4\Lib\site-packages\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [3]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline

# BCAP: choose model / run to backtest manually
RESULTS_DIR = "results/07-03-2021_17-27-59" ## PPO ONLY strategy

In [4]:
def get_daily_return(df):
    df['daily_return']=df.account_value.pct_change(1)
    #df=df.dropna()
    print('Sharpe: ',(252**0.5)*df['daily_return'].mean()/ df['daily_return'].std())
    return df

In [5]:
def backtest_strat(df):
    strategy_ret= df.copy()
    strategy_ret['Date'] = pd.to_datetime(strategy_ret['Date'])
    strategy_ret.set_index('Date', drop = False, inplace = True)
    strategy_ret.index = strategy_ret.index.tz_localize('UTC')
    del strategy_ret['Date']
    ts = pd.Series(strategy_ret['daily_return'].values, index=strategy_ret.index)
    return ts

In [6]:
def get_account_value(model_name, RESULTS_DIR):
    df_account_value=pd.DataFrame()
    for i in range(rebalance_window+validation_window, len(unique_trade_date)+1,rebalance_window):
        temp = pd.read_csv('{}/account_value_trade_{}_{}.csv'.format(RESULTS_DIR, model_name,i))
        df_account_value = df_account_value.append(temp,ignore_index=True)
    df_account_value = pd.DataFrame({'account_value':df_account_value['0']})
    sharpe=(252**0.5)*df_account_value.account_value.pct_change(1).mean()/df_account_value.account_value.pct_change(1).std()
    print(sharpe)
    df_account_value=df_account_value.join(df_trade_date[63:].reset_index(drop=True))
    return df_account_value

## DJIA

In [7]:
dji = pd.read_csv("data/^DJI.csv")
test_dji=dji[(dji['Date']>='2016-01-01') & (dji['Date']<='2020-06-30')]
test_dji = test_dji.reset_index(drop=True)

FileNotFoundError: [Errno 2] File data/^DJI.csv does not exist: 'data/^DJI.csv'

In [ ]:
dji.head()

In [ ]:
dji.tail()

In [ ]:
test_dji.shape

In [ ]:
test_dji.head()

In [ ]:
test_dji['daily_return']=test_dji['Adj Close'].pct_change(1)

In [ ]:
dow_strat = backtest_strat(test_dji)

In [ ]:
dow_strat

## Ensemble Strategy

In [ ]:
df=pd.read_csv('data/dow_30_2009_2020.csv')

In [ ]:
rebalance_window = 63
validation_window = 63
unique_trade_date = df[(df.datadate > 20151001)&(df.datadate <= 20200707)].datadate.unique()

In [ ]:
df_trade_date = pd.DataFrame({'datadate':unique_trade_date})

In [ ]:
ensemble_account_value = get_account_value('ensemble',RESULTS_DIR)

In [ ]:
ensemble_account_value.account_value.plot()

In [ ]:
ensemble_account_value = get_daily_return(ensemble_account_value)

In [ ]:
ensemble_account_value['Date'] = test_dji['Date']

In [ ]:
ensemble_account_value.head()

In [ ]:
ensemble_account_value.tail()

In [ ]:
ensemble_strat = backtest_strat(ensemble_account_value[0:1097])

In [ ]:
#ensemble_account_value.to_csv('results/df_account_value_ensemble_daily_return.csv',index=False)

In [ ]:
with pyfolio.plotting.plotting_context(font_scale=1.1):
    pyfolio.create_full_tear_sheet(returns = ensemble_strat,
                                   benchmark_rets=dow_strat, set_context=False)

#### BCAP : Discussion
so we see above that the plots are relatively weird.  
The volatility of the DRL strategy is lower than the benchmark. But could this not be because we cease trading in high turbulence periods? Need to figure out.  
  
  we also see that PPO underperforms the ensemble strategy; this makes sense by definition of the ensemble strategy, since we always pick the best-performing algorithm. If PPO was best-performing always, the ensemble would pick PPO always and we would end up at the PPO-Only strategy.  
  The only reason why the ensemble could underperform PPO-Only, would be if the performance of the agents in the 3 months validation period were not good forecasts for the future 3-month performance.
  
  This analysis needs following additions:  
  - make graphs better understandable (e.g. time stamps would be nice)
  - explain how this backtesting works exactly and what it does
  - what is "None"?
  - One plot for each agent separately what would be the performance if we would have traded on this agent only, then include in the plot the ensemble strategy (I think they have already plotted this somewhere but there is no code for this). Since the rollover is done by 3 months always (right?), would be good to indicate the 3-months ticks as well in the plot.
  - Get a metric for the following:
      - how often did the algo pick the correct agent for the subsequent trading period?
      - If an agent that has performed well in the past 3 months continues to perform well in the next 3 months, that could indicate that there is momentum in the market (?)
      - Test when the "pick" performs well and when not. Does momentum and reversal in the market have anything to do with it?
      
**ALSO**: I am not sure if they really do "backtesting" here. First, they use a period they are using for training anyway. Second, they don't use the trained model to run it again on a past timeline, but they pick the returns already calculated during the model run and basically use quantopian simply to plot against benchmark and get some metrix automatically.  
=> the fact that they don't rerun it on the backtesting period removes potential for different performance due to chance / probability, since there are randomness parts in the algorithm at multiple stages.
=> Should test on a real backtesting period, not within-training period.